# Procesado de resultados (+ logs PCSMOTE)
Notebook parametrizable para analizar múltiples **modelos** y **datasets**.

## Instrucciones
1. Editá la celda de **Configuración** con tus rutas y listas de modelos/datasets.
2. Ejecutá las celdas en orden.
3. Si activás `EXPORTAR_EXCEL`, se generará un .xlsx con hojas por (modelo,dataset) y logs encontrados.

**Convenciones de archivos**

- Resultados: `codigo/resultados/resultados_[Modelo].csv`  
- Logs PCSMOTE: `codigo/datasets/datasets_aumentados/pcsmote/logs/log_pcsmote_[dataset]_D[d]_R[r]_P[p].csv`

In [8]:
# === Configuración ===
from pathlib import Path

from pathlib import Path

# Directorio actual del notebook
BASE = Path.cwd()

# Subir un nivel (de notebooks → codigo)
ROOT = BASE.parent

PATH_RESULTADOS = ROOT / "resultados"
PATH_LOGS = ROOT / "datasets" / "datasets_aumentados" / "pcsmote" / "logs"

# Lista de modelos a estudiar (usa exactamente estos nombres para mapear a archivos):
# 'LogisticRegression' -> resultados_LogisticRegression.csv
# 'RandomForest'       -> resultados_RandomForest.csv
# 'SVM'                -> resultados_SVM.csv
MODELOS = ['LogisticRegression', 'RandomForest', 'SVM']  # editá lo que necesites

# Lista de datasets a estudiar
DATASETS = ['glass','ecoli','wdbc','heart','statlog+shuttle']  # agregá más: ['glass','ecoli','wdbc','heart','statlog+shuttle', ... ]

# Cantidad de filas top por (modelo,dataset)
TOP_N = 2

# Exportación a Excel (opcional)
EXPORTAR_EXCEL = True
RUTA_XLSX_SALIDA = ROOT / "resultados" / "estudio_de_resultados_rama-exp_foldsafe-pcsmote-gap.xlsx"


In [9]:
# === Funciones utilitarias ===
import os,json, re
import pandas as pd
from collections import defaultdict
from IPython.display import display

PREFERENCIAS_METRICAS = [
    'gmean_macro', 'f1_macro', 'roc_auc_macro_ovr',
    'balanced_accuracy', 'kappa', 'mcc', 'accuracy', 'score'
]

# Acumuladores
bloques_por_modelo: dict[str, list[tuple[str, pd.DataFrame]]] = defaultdict(list)
logs_encontrados: list[tuple[str, pd.DataFrame]] = []

# --- Función para ajustar nombres de hoja ---
nombres_usados = set()

def alias_modelo(nombre: str) -> str:
    s = normalizar_str(nombre)
    if s == "randomforest":
        return "RF"
    if s == "logisticregression":
        return "LR"
    return nombre  # SVM u otros

def ajustar_nombre(nombre: str) -> str:
    nombre = nombre[:31]  # Excel no admite > 31 caracteres
    if nombre in nombres_usados:
        base = nombre[:28]  # dejar espacio para sufijo
        i = 1
        nuevo = f"{base}{i}"
        while nuevo in nombres_usados:
            i += 1
            nuevo = f"{base}{i}"
        nombre = nuevo
    nombres_usados.add(nombre)
    return nombre

def normalizar_str(x: str) -> str:
    return str(x).strip().lower()

def cargar_resultados(path_resultados: Path, modelos: list[str]) -> pd.DataFrame:
    dfs = []
    for m in modelos:
        ruta = path_resultados / f"resultados_{m}.csv"
        if not ruta.exists():
            print(f"[!] No encontré {ruta}, continúo…")
            continue
        try:
            df = pd.read_csv(ruta, encoding="utf-8")
        except UnicodeDecodeError:
            df = pd.read_csv(ruta, encoding="latin1")
        df.columns = [str(c).strip().replace(" ","_").replace("-","_").replace("/","_").lower() for c in df.columns]
        if "modelo" not in df.columns:
            df["modelo"] = m
        dfs.append(df)
    if not dfs:
        raise FileNotFoundError("No se cargó ningún CSV.")
    return pd.concat(dfs, ignore_index=True)

def elegir_metrica(resultados):
    dataset = resultados['dataset']
    densidad = resultados['densidad']
    riesgo = resultados['riesgo']
    pureza = resultados['pureza']
    
    for i in range(len(dataset)):
        archivo_log = f"log_pcsmote_{dataset.iloc[i]}_D{densidad.iloc[i]}_R{riesgo.iloc[i]}_P{pureza.iloc[i]}.csv"
        archivo_log_path = os.path.join(PATH_LOGS, archivo_log)
        
        # Verificar si el archivo existe
        if os.path.exists(archivo_log_path):
            print(f"Encontrado archivo de log: {archivo_log_path}")
        else:
            print(f"No encontrado archivo de log: {archivo_log_path}")
    
    return 'cv_f1_macro'

def col_dataset_name(df: pd.DataFrame) -> str:
    for cand in ('dataset','nombre_dataset','dataset_name','ds','nombre'):
        if cand in df.columns:
            return cand
    raise ValueError("No encuentro columna de dataset ('dataset' o 'nombre_dataset').")

def top_n_por_modelo_dataset(df: pd.DataFrame, modelo: str, dataset: str, metrica: str | None, n: int=2) -> pd.DataFrame:
    sub = df[df['modelo'].map(normalizar_str) == normalizar_str(modelo)].copy()
    cd = col_dataset_name(df)
    sub = sub[sub[cd].map(normalizar_str) == normalizar_str(dataset)]
    if sub.empty:
        return sub
    if metrica is None or metrica not in sub.columns:
        return sub.head(n)
    return sub.sort_values(metrica, ascending=False).head(n)

def detectar_pcsmote(row: pd.Series) -> bool:
    for c in ['tecnica','sobremuestreo','oversampler','sampler','tecnica_sobremuestreo']:
        if c in row.index and isinstance(row[c], str) and row[c] and row[c].strip().lower() == 'pcsmote':
            return True
    for c in ['dataset_file','mejor_configuracion','config']:
        if c in row.index and isinstance(row[c], str) and 'pcsmote' in row[c].lower():
            return True
    return False

def limpiar_num(x: str) -> str:
    """
    Convierte strings tipo '25.0' en '25', deja otros valores igual.
    """
    try:
        f = float(x)
        if f.is_integer():
            return str(int(f))
        return str(f)
    except:
        return str(x).strip()

def extraer_drps_desde_fila(row: pd.Series):
    # 1) columnas explícitas
    d = str(row.get('densidad','')).strip() or None
    r = str(row.get('riesgo','')).strip() or None
    p = str(row.get('pureza','')).strip() or None
    if d and r and p:
        return limpiar_num(d), limpiar_num(r), limpiar_num(p)

    # 2) JSON o texto en mejor_configuracion
    txt = str(row.get('mejor_configuracion','') or '')
    if txt:
        try:
            j = json.loads(txt)
            if isinstance(j, dict) and 'raw' in j and isinstance(j['raw'], str):
                try:
                    j2 = json.loads(j['raw'].replace("'", '"'))
                    j = j2
                except Exception:
                    pass
            d = j.get('densidad') or j.get('D') or j.get('densidad_pct')
            r = j.get('riesgo')   or j.get('R')
            p = j.get('pureza')   or j.get('P')
            if d and r and p:
                return str(d), str(r), str(p)
        except Exception:
            pass
        # Regex fallback
        mD = re.search(r"_D([^_]+)", txt)
        mR = re.search(r"_R([^_]+)", txt)
        mP = re.search(r"_P([^_./\\]+)", txt)
        if mD and mR and mP:
            return mD.group(1), mR.group(1), mP.group(1)

    # 3) dataset_file del estilo pcsmote_glass_D25_R25_Pproporcion_train.csv
    for c in ['dataset_file','train_file','test_file','archivo']:
        val = row.get(c)
        if isinstance(val, str) and val:
            mD = re.search(r"_D([^_]+)", val)
            mR = re.search(r"_R([^_]+)", val)
            mP = re.search(r"_P([^_./\\]+)", val)
            if mD and mR and mP:
                return mD.group(1), mR.group(1), mP.group(1)
    return None

def buscar_log_pcsmote(path_logs: Path, dataset: str, d: str, r: str, p: str):
    target = f"log_pcsmote_{dataset}_D{d}_R{r}_P{p}.csv".lower()
    if not path_logs.exists():
        return None
    for f in path_logs.rglob('*.csv'):
        if f.name.lower() == target:
            return f
    return None


In [10]:
# === Carga de resultados y selección de métrica ===
resultados = cargar_resultados(PATH_RESULTADOS, MODELOS)
METRICA = elegir_metrica(resultados)
print('Métrica elegida para ordenar (desc):', METRICA)

# Inferimos columna dataset una vez
COL_DATASET = None
for cand in ('dataset','nombre_dataset','dataset_name','ds','nombre'):
    if cand in resultados.columns:
        COL_DATASET = cand
        break
if COL_DATASET is None:
    raise ValueError("No encuentro columna de dataset ('dataset' o 'nombre_dataset').")

resultados.head(3)  # preview


Encontrado archivo de log: d:\Documentos_D\TESIS\armado tesina\codigo\datasets\datasets_aumentados\pcsmote\logs\log_pcsmote_glass_D75_R75_Pentropia.csv
Encontrado archivo de log: d:\Documentos_D\TESIS\armado tesina\codigo\datasets\datasets_aumentados\pcsmote\logs\log_pcsmote_heart_D75_R25_Pentropia.csv
Encontrado archivo de log: d:\Documentos_D\TESIS\armado tesina\codigo\datasets\datasets_aumentados\pcsmote\logs\log_pcsmote_shuttle_D25_R25_Pentropia.csv
No encontrado archivo de log: d:\Documentos_D\TESIS\armado tesina\codigo\datasets\datasets_aumentados\pcsmote\logs\log_pcsmote_wdbc_D75_R75_Pentropia.csv
Encontrado archivo de log: d:\Documentos_D\TESIS\armado tesina\codigo\datasets\datasets_aumentados\pcsmote\logs\log_pcsmote_glass_D50_R50_Pentropia.csv
Encontrado archivo de log: d:\Documentos_D\TESIS\armado tesina\codigo\datasets\datasets_aumentados\pcsmote\logs\log_pcsmote_heart_D25_R25_Pentropia.csv
Encontrado archivo de log: d:\Documentos_D\TESIS\armado tesina\codigo\datasets\datas

,dataset,tipo,tecnica,densidad,riesgo,pureza,n_train,n_test,n_features,es_grande,...,cv_balanced_accuracy,cv_mcc,cv_cohen_kappa,test_f1_macro,test_balanced_accuracy,test_mcc,test_cohen_kappa,gap_f1_macro,search_time_sec,n_jobs_search
0,glass,foldsafe,pcsmote,75,75,entropia,149,65,9,False,...,0.591953,0.385884,0.360361,0.606281,0.738716,0.502483,0.474140,-0.077738,4.716,4
1,heart,foldsafe,pcsmote,75,25,entropia,207,90,13,False,...,0.362892,0.323107,0.317696,0.377129,0.423106,0.377547,0.370166,-0.030087,5.889,4
2,shuttle,foldsafe,pcsmote,25,25,entropia,40600,17400,9,True,...,0.912039,0.621753,0.607459,0.746799,0.917848,0.622590,0.606018,-0.002626,2054.510,1


In [11]:
# === Estudio por lotes (MODELOS x DATASETS) ===
from IPython.display import display
import pandas as pd

# Acumuladores para exportar
hojas_excel: dict[str, pd.DataFrame] = {}
logs_encontrados: list[tuple[str, pd.DataFrame]] = []

for modelo in MODELOS:
    for ds in DATASETS:
        print('='*90)
        print(f'Modelo: {modelo} | Dataset: {ds}')

        topN = top_n_por_modelo_dataset(resultados, modelo, ds, METRICA, n=TOP_N)

        # Fila base para este modelo/dataset
        base_row = resultados[
            (resultados['modelo'].map(normalizar_str) == normalizar_str(modelo)) &
            (resultados[COL_DATASET].map(normalizar_str) == normalizar_str(ds)) &
            (resultados['tipo'].map(normalizar_str) == 'base')
        ]

        # Concatenar topN + base (si existe)
        combinado = pd.concat([topN, base_row], ignore_index=True) if not base_row.empty else topN

        if combinado.empty:
            print(' -> Sin filas para esta combinación.')
            continue

        display(combinado)

        # Guardar bloque en la hoja del MODELO (una hoja por modelo; múltiples datasets)
        bloques_por_modelo[modelo].append((ds, combinado))

        # ---- Logs PCSMOTE (igual que antes) ----
        for idx, row in topN.iterrows():
            if detectar_pcsmote(row):
                drp = extraer_drps_desde_fila(row)
                if not drp:
                    print(f'   [Fila {idx}] PCSMOTE detectado pero no pude extraer (D,R,P).')
                    continue
                d, r, p = drp
                ds_norm = str(row[COL_DATASET]).strip().lower()
                log_path = buscar_log_pcsmote(PATH_LOGS, ds_norm, str(d), str(r), str(p))
                if log_path is None or not log_path.exists():
                    print(f'   [!] No se encontró log: log_pcsmote_{ds_norm}_D{d}_R{r}_P{p}.csv en {PATH_LOGS}')
                    continue
                try:
                    log_df = pd.read_csv(log_path)
                    print(f'   [+] Log leído: {log_path.name}  (filas={len(log_df)})')
                    display(log_df.head(10))

                    mstr = normalizar_str(modelo)
                    modelo_norm = "RF" if mstr == "randomforest" else ("LR" if mstr == "logisticregression" else modelo)
                    hoja_nombre = f'log_{modelo_norm}_{ds_norm}_{d}_{r}_{p}'

                    logs_encontrados.append((hoja_nombre, log_df))
                except Exception as e:
                    print(f'   [x] Error leyendo {log_path.name}: {e}')

Modelo: LogisticRegression | Dataset: glass


,dataset,tipo,tecnica,densidad,riesgo,pureza,n_train,n_test,n_features,es_grande,...,cv_balanced_accuracy,cv_mcc,cv_cohen_kappa,test_f1_macro,test_balanced_accuracy,test_mcc,test_cohen_kappa,gap_f1_macro,search_time_sec,n_jobs_search
0,glass,foldsafe,pcsmote,75,75,entropia,149,65,9,False,...,0.591953,0.385884,0.360361,0.606281,0.738716,0.502483,0.47414,-0.077738,4.716,4


   [+] Log leído: log_pcsmote_glass_D75_R75_Pentropia.csv  (filas=6)


,dataset,clase,train_original,objetivo_balance,estado,motivo_sin_sinteticas,faltante_solicitado,faltante_final,tope_por_clase_aplicado,tope_global_aplicado,...,umbral_densidad,umbral_entropia,percentil_densidad,percentil_riesgo,criterio_pureza,tecnica_sobremuestreo,factor_equilibrio,random_state,modo_espacial,timestamp
0,glass,1,56,48,no se sobremuestrea,NaN,0,0,False,False,...,NaN,NaN,75.0,75.0,entropia,PCSMOTE,0.8,42,2d,2025-09-11T15:33:55.208891
1,glass,2,61,48,no se sobremuestrea,NaN,0,0,False,False,...,NaN,NaN,75.0,75.0,entropia,PCSMOTE,0.8,42,2d,2025-09-11T15:33:55.208891
2,glass,3,14,48,sobremuestreada,ok,34,34,False,False,...,1.0,0.721928,75.0,75.0,entropia,PCSMOTE,0.8,42,2d,2025-09-11T15:33:55.230034
3,glass,5,10,48,sobremuestreada,insuficientes_filtradas(<k+1),38,38,False,False,...,0.2,0.970951,75.0,75.0,entropia,PCSMOTE,0.8,42,2d,2025-09-11T15:33:55.241210
4,glass,6,7,48,sobremuestreada,insuficientes_filtradas(<k+1),41,41,False,False,...,0.6,0.970951,75.0,75.0,entropia,PCSMOTE,0.8,42,2d,2025-09-11T15:33:55.250213
5,glass,7,23,48,sobremuestreada,ok,25,25,False,False,...,1.0,0.000000,75.0,75.0,entropia,PCSMOTE,0.8,42,2d,2025-09-11T15:33:55.277302


Modelo: LogisticRegression | Dataset: ecoli
 -> Sin filas para esta combinación.
Modelo: LogisticRegression | Dataset: wdbc


...

   [!] No se encontró log: log_pcsmote_wdbc_D50_R50_Pentropia.csv en d:\Documentos_D\TESIS\armado tesina\codigo\datasets\datasets_aumentados\pcsmote\logs
Modelo: SVM | Dataset: heart


,dataset,tipo,tecnica,densidad,riesgo,pureza,n_train,n_test,n_features,es_grande,...,cv_balanced_accuracy,cv_mcc,cv_cohen_kappa,test_f1_macro,test_balanced_accuracy,test_mcc,test_cohen_kappa,gap_f1_macro,search_time_sec,n_jobs_search
9,heart,foldsafe,pcsmote,75,75,entropia,207,90,13,False,...,0.360741,0.282904,0.275525,0.346805,0.354924,0.330538,0.321243,-0.009937,0.968,4


   [+] Log leído: log_pcsmote_heart_D75_R75_Pentropia.csv  (filas=5)


,dataset,clase,train_original,objetivo_balance,estado,motivo_sin_sinteticas,faltante_solicitado,faltante_final,tope_por_clase_aplicado,tope_global_aplicado,...,umbral_densidad,umbral_entropia,percentil_densidad,percentil_riesgo,criterio_pureza,tecnica_sobremuestreo,factor_equilibrio,random_state,modo_espacial,timestamp
0,heart,0,128,102,no se sobremuestrea,NaN,0,0,False,False,...,NaN,NaN,75.0,75.0,entropia,PCSMOTE,0.8,42,2d,2025-09-11T15:33:59.354217
1,heart,1,43,102,sobremuestreada,ok,59,59,False,False,...,0.1,0.970951,75.0,75.0,entropia,PCSMOTE,0.8,42,2d,2025-09-11T15:33:59.506880
2,heart,2,28,102,sobremuestreada,ok,74,74,False,False,...,0.0,0.970951,75.0,75.0,entropia,PCSMOTE,0.8,42,2d,2025-09-11T15:33:59.584361
3,heart,3,28,102,sobremuestreada,ok,74,74,False,False,...,0.0,0.970951,75.0,75.0,entropia,PCSMOTE,0.8,42,2d,2025-09-11T15:33:59.731103
4,heart,4,10,102,sobremuestreada,ok,92,92,False,False,...,0.0,0.541446,75.0,75.0,entropia,PCSMOTE,0.8,42,2d,2025-09-11T15:33:59.819600


Modelo: SVM | Dataset: statlog+shuttle
 -> Sin filas para esta combinación.


In [12]:
import pandas as pd

EXPORTAR_EXCEL = True

if EXPORTAR_EXCEL:
    RUTA_XLSX_SALIDA.parent.mkdir(parents=True, exist_ok=True)
    with pd.ExcelWriter(RUTA_XLSX_SALIDA, engine="xlsxwriter") as xw:
        # Formatos
        fmt_border = xw.book.add_format({'border': 1})
        fmt_wrap   = xw.book.add_format({'text_wrap': True, 'valign': 'top', 'border': 1})
        fmt_title  = xw.book.add_format({'bold': True})

        # ===== Hojas TOP por MODELO (una hoja por modelo; múltiples datasets) =====
        for modelo, bloques in bloques_por_modelo.items():
            sheet_name = ajustar_nombre(f"top+{alias_modelo(modelo)}")
            # Worksheet se crea al primer to_excel
            # Para calcular anchos: concateno todos los bloques (solo para sizing)
            df_all = pd.concat([df for _, df in bloques], ignore_index=True) if bloques else pd.DataFrame()

            start_row = 0
            for ds, df in bloques:
                # Rótulo de dataset
                ws = xw.book.add_worksheet(sheet_name) if sheet_name not in xw.sheets else xw.sheets[sheet_name]
                if start_row == 0 and sheet_name not in xw.sheets:
                    # si recién se creó, registrar
                    xw.sheets[sheet_name] = ws
                else:
                    ws = xw.sheets[sheet_name]

                ws.write(start_row, 0, f"Dataset: {ds}", fmt_title)
                start_row += 1

                # Escribir bloque con cabecera
                df.to_excel(xw, sheet_name=sheet_name, startrow=start_row, index=False, header=True)
                start_row += len(df) + 2  # una fila en blanco entre bloques

            # Ajustes de columnas (sobre df_all)
            if not df_all.empty:
                ws = xw.sheets[sheet_name]
                # Columna N (índice 13)
                ws.set_column(13, 13, 38, fmt_wrap)
                # Resto auto-ajuste + borde
                for idx, col in enumerate(df_all.columns):
                    if idx == 13:
                        continue
                    max_len = max([len(str(col))] + [len(str(v)) for v in df_all[col].astype(str)])
                    ancho = min(max_len + 2, 50)
                    ws.set_column(idx, idx, ancho, fmt_border)

        # ===== Hojas de LOGS (misma lógica que ya usabas) =====
        for name, logdf in logs_encontrados:
            sheet = ajustar_nombre(name)
            logdf.to_excel(xw, index=False, sheet_name=sheet)
            ws = xw.sheets[sheet]
            for idx, col in enumerate(logdf.columns):
                max_len = max([len(str(col))] + [len(str(v)) for v in logdf[col].astype(str)])
                ancho = min(max_len + 2, 50)
                ws.set_column(idx, idx, ancho, fmt_border)

    print(f"[i] Exportado a: {RUTA_XLSX_SALIDA.resolve()}")
else:
    print("Exportación a Excel desactivada (EXPORTAR_EXCEL=False).")

[i] Exportado a: D:\Documentos_D\TESIS\armado tesina\codigo\resultados\estudio_de_resultados_rama-exp_foldsafe-pcsmote-gap.xlsx


In [13]:
# import os, re, ast, json
# import pandas as pd

# excel_path = "../resultados/estudio_de_resultados.xlsx"
# out_path   = "../resultados/mapping_mejores_hparams_pipeline1.json"

# def clean_np_wrappers(txt: str) -> str:
#     return re.sub(r'(?:np|numpy)\.(?:float|int)(?:32|64)?\(\s*([^)]+?)\s*\)', r'\1', txt)

# def parse_best_params(cell):
#     if cell is None or (isinstance(cell, float) and pd.isna(cell)):
#         return None
#     t = str(cell)
#     t = re.sub(r'OrderedDict\((.*)\)$', r'\1', t)
#     t = clean_np_wrappers(t)
#     try:
#         return ast.literal_eval(t)
#     except Exception:
#         return None

# mapping = {}

# for sh in ["top+LR", "top+RF", "top+SVM"]:
#     df = pd.read_excel(excel_path, sheet_name=sh, header=None)
#     i = 0
#     while i < len(df):
#         # buscar fila con encabezados reales ("dataset", "modelo", "mejor configuracion")
#         if isinstance(df.iloc[i,0], str) and df.iloc[i,0].strip().lower() == "dataset":
#             header = df.iloc[i].tolist()
#             i += 1
#             start = i
#             # datos hasta próxima sección "Dataset:" o nueva cabecera o fila vacía
#             while i < len(df):
#                 v0 = df.iloc[i,0]
#                 if (isinstance(v0, str) and (v0.startswith("Dataset:") or v0.strip().lower()=="dataset")) or pd.isna(v0):
#                     break
#                 i += 1
#             block = df.iloc[start:i].copy()
#             block.columns = header

#             # nombres normalizados en minúscula
#             lower_cols = {c.lower().strip(): c for c in block.columns if isinstance(c,str)}
#             def col(name):
#                 # admite "mejor configuracion" (con espacio) y variantes
#                 alias = {
#                     "mejor_configuracion": ["mejor configuracion", "best_params", "mejor_parametros", "mejor parametros"]
#                 }
#                 if name == "mejor_configuracion":
#                     for v in [name] + alias[name]:
#                         if v in lower_cols:
#                             return lower_cols[v]
#                 return lower_cols.get(name, None)

#             c_modelo  = col("modelo")
#             c_dataset = col("dataset")
#             c_tipo    = col("tipo")
#             c_best    = col("mejor_configuracion")

#             if not (c_modelo and c_dataset and c_best):
#                 continue

#             for _, r in block.iterrows():
#                 modelo  = str(r.get(c_modelo, "")).strip()
#                 dataset = str(r.get(c_dataset, "")).strip()
#                 tipo    = str(r.get(c_tipo, "")).strip() if c_tipo else ""
#                 best    = r.get(c_best, None)
#                 best_d  = parse_best_params(best)
#                 if not modelo or not dataset or not isinstance(best_d, dict):
#                     continue
#                 clf_params = {
#                     k.replace("classifier__", "clf__"): v
#                     for k, v in best_d.items()
#                     if isinstance(k, str) and (k.startswith("clf__") or k.startswith("classifier__"))
#                 }
#                 if clf_params:
#                     mapping[str((modelo, dataset, tipo))] = clf_params
#         else:
#             i += 1

# os.makedirs(os.path.dirname(out_path), exist_ok=True)
# with open(out_path, "w", encoding="utf-8") as f:
#     json.dump(mapping, f, ensure_ascii=False, indent=2)

# print(f"✅ Mapping generado: {out_path}")
# print(f"   Entradas mapeadas: {len(mapping)}")
